In [1]:
! pip install pandas
! pip install tqdm
! pip install langchain_huggingface
! pip install langchain
! pip install langchain_community
! pip install faiss-cpu
! pip install tf-keras

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached numpy-2.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.

In [2]:
! pip install markdown
! pip install requests 
! pip install markitdown
! pip install langchain-gigachat

Could not fetch URL https://pypi.org/simple/langchain-gigachat/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/langchain-gigachat/ (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)'))) - skipping
ERROR: Could not find a version that satisfies the requirement langchain-gigachat (from versions: none)
ERROR: No matching distribution found for langchain-gigachat


In [ ]:
from openai import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
# ollama run gemma2
llm_client = OpenAI(
    base_url='http://localhost:8000/v1',
    api_key="EMPTY"
)

model_name="deepvk/USER-bge-m3"
model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {'normalize_embeddings': False}
model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

pdf to md

In [ ]:
from markitdown import MarkItDown
import os
import json

inpt = 'input_docs'
output_dir = 'output_docs'

md = MarkItDown()
files = [f for f in os.listdir(inpt) if os.path.isfile(os.path.join(inpt, f))]

for i in files:
    result = md.convert(f"{inpt}/{i}")
    with open(output_dir+'/' + i.replace('.pdf', '.md'), 'w') as f:
        f.write(result.text_content)
print('done')

done


In [ ]:
import re
import os
CHUNKS = {}

files = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]

for i in files:
    with open(output_dir + '/' + i, 'r') as file:
        CHUNKS[i] = file.read().split('')[1::]


Сборка векторной базы данных

In [6]:
import pandas as pd
from langchain.schema import Document
import numpy as np
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.docstore.document import Document
import tqdm

In [7]:


rows = []
for source_file, chunk in CHUNKS.items():
    for chunk_text in chunk:
        rows.append({'source_file': source_file, 'chunk_text': chunk_text})

df = pd.DataFrame(rows)

print(len(max(df['chunk_text'], key=len)))


7091


In [8]:
def preprocess_text(text):
    text = re.sub(r'[0-9\s]', ' ', text)
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', ' ', text) 
    text = re.sub(r'\s+', ' ', text).strip()  
    
    tokens = text.lower().split()
    cleaned_text = ' '.join(tokens)

    return cleaned_text

def dataframe_preprocess(df):
    try:
        df = df.drop(columns=['Unnamed: 0'])
    except Exception:
        pass
    df = df.dropna().reset_index(drop=True)

    # очистка текста + дедубликация
    df['preprocessed_text'] = df['chunk_text'].apply(lambda x: preprocess_text(x))

    df['embedded_text'] = df['preprocessed_text'].apply(lambda x: model.embed_query(x))

    return df
ppc_df = dataframe_preprocess(df)
ppc_df

,source_file,chunk_text,preprocessed_text,embedded_text
0,Кртакое_руководство_по_разбрасователю_Амазон_Z...,Прицепные распределители ZG-TS и ZG-B\n\nИдеал...,прицепные распределители zg ts и zg b идеально...,"[-0.01828356832265854, -0.018634775653481483, ..."
1,Кртакое_руководство_по_разбрасователю_Амазон_Z...,2\n\n3\n\nБеспрепятственный рост растений явля...,беспрепятственный рост растений является важне...,"[-0.018500659614801407, -0.01784105971455574, ..."
2,Кртакое_руководство_по_разбрасователю_Амазон_Z...,Прицепной распределитель ZG-TS\n\n4\n\nот 15 м...,прицепной распределитель zg ts от м до м л или...,"[-0.03543168306350708, 0.01444028876721859, -0..."
3,Кртакое_руководство_по_разбрасователю_Амазон_Z...,Прицепной распределитель ZG-TS\n\nВаши преимущ...,прицепной распределитель zg ts ваши преимущест...,"[-0.03505029156804085, -0.044518426060676575, ..."
4,Кртакое_руководство_по_разбрасователю_Амазон_Z...,ZG-TS –\nМасштаб прецизионности\n\n6\n\nС Wind...,zg ts масштаб прецизионности с windcontrol мож...,"[-0.06268981099128723, -0.003406390780583024, ..."
...,...,...,...,...
517,Руководство_по_разбрасователю_Амазон_ZG_TS_750...,"12.13 Проверка масляного фильтра\n\nОчистка, ...",проверка масляного фильтра очистка техническое...,"[-0.05273043364286423, -0.0655040368437767, -0..."
518,Руководство_по_разбрасователю_Амазон_ZG_TS_750...,"Очистка, техническое обслуживание и ремонт\n\n...",очистка техническое обслуживание и ремонт заме...,"[-0.032747432589530945, -0.022843530401587486,..."
519,Руководство_по_разбрасователю_Амазон_ZG_TS_750...,"12.18 Моменты затяжки болтов\n\nОчистка, техн...",моменты затяжки болтов очистка техническое обс...,"[0.02803521789610386, 0.010819952003657818, -0..."
520,Руководство_по_разбрасователю_Амазон_ZG_TS_750...,Гидравлическая схема\n\n13\n\nГидравлическая с...,гидравлическая схема гидравлическая схема zg t...,"[-0.0006461550947278738, 0.031581051647663116,..."


In [9]:

def VB_build(df):
    sample_text = "Sample text."
    embedding = model.embed_query(sample_text)

    dimension = len(embedding)
    index = faiss.IndexFlatIP(dimension)
    df['embedded_text'] = df['embedded_text'].apply(lambda x: np.array(x))

    embeddings = np.stack(df['embedded_text'].tolist())

    index.add(embeddings)

    documents = [
        Document(page_content=text, metadata={'source_file': sf})
        for text, sf in zip(df['chunk_text'], 
                                         df['source_file'])
    ]
    
    docstore = InMemoryDocstore(dict(enumerate(documents)))
    
    vector_store = FAISS(
        embedding_function=model,
        index=index,
        docstore=docstore,
        index_to_docstore_id=dict(zip(range(len(embeddings)), range(len(embeddings)))),
    )

    # Save the vector database and additional info
    vector_store.save_local('vector_database/vector_database')
    df.to_csv('vector_database/docs_info.csv', index=False)

    return vector_store

VB_build(ppc_df)


In [10]:
def search_VB(query, k=5):
    vector_store = FAISS.load_local(
        'vector_database/vector_database',
        embeddings=model,
        allow_dangerous_deserialization=True
    )
    return vector_store.similarity_search(query, k=k)

In [48]:
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from typing import List

def send_req(user_input: str, documents: List[Document]) -> str:
    with open('prompt.txt', 'r') as file:
        GROUNDED_SYSTEM_PROMPT = file.read()
    sample_history = [
        {'role': 'system', 'content': GROUNDED_SYSTEM_PROMPT}, 
        {'role': 'documents', 'content': json.dumps(documents, ensure_ascii=False)},
        {'role': 'user', 'content': user_input}
    ]
    
    relevant_indexes = llm_client.chat.completions.create(
        messages=sample_history,
        model='Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24',
        temperature=0.0
    ).choices[0].message.content

    final_answer = llm_client.chat.completions.create(
        model='Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24',
        messages=sample_history + [{'role': 'assistant', 'content': relevant_indexes}],
        temperature=0.0
    ).choices[0].message.content
    
    return final_answer


In [ ]:

def dump_to_json(x: Document, i: int):
    x = x.model_dump()
    x['id'] = i
    return x

subm = list()
for row in pd.read_csv("test.csv")['user_input'].to_list():
    documents = [dump_to_json(doc, i) for i, doc in enumerate(search_VB(row, k=5))]
    answer = send_req(row, documents)
    documents = [i["page_content"] for i in documents]
    print({"contexts": documents, "response": answer})
    subm.append({"contexts": documents, "response": answer})



{'contexts': ['2\n\n3\n\nБеспрепятственный рост растений является важнейшим условием для высокой урожайности. Но условия для роста не везде\nодинаковые. Поэтому подкормка всегда должна быть сбалансированной и отвечать потребностям в минеральных удобрениях\nименно для данной почвы. Решающим фактором в достижении успеха является не только правильный выбор удобрения,\nно прежде всего эффективное и аккуратное распределение питательных веществ.\n\nZG-TS\n\nПрецизионное внесение удобрений,\nболее высокая урожайность\n\nСтраница\n\nПрофессиональный распределитель\nминеральных удобрений\n\nZG-TS | Ваши преимущества\n\nZG-TS | Новый масштаб\n\nМодель\n\nZG-TS 7501\n\nZG-TS 10001\n\nОбъем бункера\n\n7.500 л\n\n10.000 л\n\nZG-TS Truck\n\nZG-TS | Высококачественное многослойное лакирование\n\nZG-TS | Бункер\n\nZG-TS | Руление I Регулировка тормозного усилия\n\nZG-TS | Взвешивающая техника ProfisPro\n\nZG-TS | Интеллектуальный менеджмент заполнения\n\nZG-TS | Привод распределительных дисков\n\nZG-T

In [ ]:
pd.DataFrame(subm)

In [61]:
pd.DataFrame(subm).to_csv('submission.csv', index=False)

In [64]:
import pandas as pd
data = pd.read_csv('submission2.csv', encoding='utf-8')

In [65]:
data

,contexts,response
0,['2\n\n3\n\nБеспрепятственный рост растений яв...,Для настройки нормы внесения удобрений на расп...
1,['Общие правила техники безопасности\n\n20\n\n...,Перед первым вводом в эксплуатацию агрегата ZG...
2,['30\n\nAutoTS\n\nИнтегрированная в диски\nсис...,Система AutoTS представляет собой интегрирован...
3,['Содержание\n\n12.13\n12.14\n12.15\n12.16\n12...,Для проверки масляного фильтра распределителя ...
4,['ZG-TS | Технология пограничного распределени...,Для настройки пограничного распределения с исп...
...,...,...
110,['Эксплуатация агрегата\n\n (2) Разбрасывание...,Для настройки трактора в соответствии с параме...
111,['10 | Настройка GPS\nНастройка приемника Ag-S...,"К сожалению, я не могу найти запрашиваемую инф..."
112,['Регулировка\n\nВсегда заменяйте короткий и д...,"К сожалению, я не могу найти запрашиваемую инф..."
113,['3. Чтобы выбрать данные задания для\n\nимпор...,Для загрузки данных задания через приложение m...


In [70]:
data.to_csv('submission.csv', encoding='utf-8', index=False)

fappy

In [3]:
! pip install fastapi uvicorn langchain faiss-cpu numpy pydantic transformers


Could not fetch URL https://pypi.org/simple/fastapi/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/fastapi/ (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)'))) - skipping
ERROR: Could not find a version that satisfies the requirement fastapi (from versions: none)
ERROR: No matching distribution found for fastapi
